In [1]:
from pathlib import Path

import pandas as pd

from portfolio_factor_allocation.backtesting import block_bootstrap_metrics

In [2]:
data_dir = Path.cwd().parent / "data"
returns_dir = data_dir / "returns"
bootstrap_dir = data_dir / "bootstrap"

(bootstrap_dir / "sharpe").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "volatility").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "information_ratio").mkdir(parents=True, exist_ok=True)
(bootstrap_dir / "tracking_error").mkdir(parents=True, exist_ok=True)

### Load data

### Block bootstrapping

In [3]:
factor_combs = [["V", "W"]]
suffixes = [
    "_int_ter",
    "_mix_ter",
    "_int_dec",
    "_mix_dec",
    "_int_bw",
    "_mix_bw",
]
block_size = 5  # In months
n_sim = 1000
r_f = 0.0
seed = 42

In [4]:
# Load market returns
df_market = pd.read_csv(returns_dir / "market.csv")
df_market["date"] = pd.to_datetime(df_market["date"]).dt.to_period("M")
df_market = df_market.set_index("date")

for factor_comb in factor_combs:
    factor_str = "_".join(factor_comb)
    returns_cols = [factor_str + s for s in suffixes]
    df_r = pd.read_csv(returns_dir / f"{factor_str}.csv")
    df_r["date"] = pd.to_datetime(df_r["date"]).dt.to_period("M")
    df_r = df_r.set_index("date")

    # Align
    aligned_r, aligned_m = df_r.align(df_market, join="inner", axis=0)

    # Select only the relevant columns
    df_r_subset = aligned_r[returns_cols]

    # Perform block bootstrapping
    results = block_bootstrap_metrics(
        df_r_subset, aligned_m["market"], block_size, n_sim, r_f=r_f, seed=seed
    )

    # Save results
    results["sharpe"].to_csv(
        bootstrap_dir / "sharpe" / f"{factor_str}.csv", index=False
    )
    results["volatility"].to_csv(
        bootstrap_dir / "volatility" / f"{factor_str}.csv", index=False
    )
    results["information_ratio"].to_csv(
        bootstrap_dir / "information_ratio" / f"{factor_str}.csv", index=False
    )
    results["tracking_error"].to_csv(
        bootstrap_dir / "tracking_error" / f"{factor_str}.csv", index=False
    )